# BAI-Assistent

Grundidee: Hilft bei der Erstellung von Zusammenfassungen, welche auf Basis unseren Zusammenfassungen und Vorlesungsfolien die Antworten generiert

#### Unsere Problemstellung
Während der Prüfungsvorbereitung sind vor allem Erstsemester Studenten überfordert, wie man beim Lernen vorgehen kann. Daher haben wir es als Lücke vor allem im BAI-Studiengang erkannt.

#### Use Cases: 
Ich möchte, dass mir der Lernassistent mir Fachbegriffe in ML und Einführung KI erklärt
Ich möchte gut auf die Prüfungen durch den Lernassistenten vorbereitet werden
-	Ich möchte Prüfungsfragen erhalten
-	Ich möchte, dass es Merksätze gibt
-	Ich möchte, dass die Erklärungen einfach sind
-	Ich möchte Hilfe/Beratung erhalten, wie ich mein Cheat Sheet gemäss Stoffabgrenzung aufstellen kann
Ich möchte schnelle und unlimitierte Antworten
(Ich möchte Prüfungsfragen vom Chatbot erhalten, damit ich mich gut auf die Prüfung vorbereiten kann)

##### Zielgruppe: 
BAI-Studenten im ersten Studienjahr, die Maschinelles Lernen und Einführung in die Künstliche Intelligenz belegen

##### KPIs: 
•	Antwortzeit < 5 Sekunden
•	Prüfungsnutzen > 70 % finden Quiz hilfreich
•	Fachliche Korrektheit >85%

Unsere Erwartungen: Fachbegriffe fragen, Unterschied zwischen Supervised und Unsupervised Learning, Was ist One Hot Encoding
Inhalte für den KI-Assistenten: Folien Unterricht, Zusammenfassungen, Stoffabgrenzung


#### Zusammenführung LLM und API

In [ ]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

# --- Stelle sicher, dass dieser Key existiert ---
#assert "GROQ_API_KEY" in os.environ, "GROQ_API_KEY fehlt in den Env Vars!"

# --- Initialisiere LLM explizit für GROQ ---
# llm = ChatOpenAI(  #Groq Verbindung
#     model="openai/gpt-oss-120b",   
#     api_key=os.environ["GROQ_API_KEY"],
#     base_url="https://api.groq.com/openai/v1",
#     temperature=0.3,
# )

# --- Initialisiere LLM explizit für CEREBRAS ---
assert "CEREBRAS_API_KEY" in os.environ, "CEREBRAS_API_KEY fehlt in den Env Vars!"

llm = ChatOpenAI( #Cerebras Verbindung
    model="gpt-oss-120b",   
    api_key=os.environ["CEREBRAS_API_KEY"],
    base_url="https://api.cerebras.ai/v1",
    temperature=0.3,
)

print("Sende Test-Ping...")
try:
    msg = llm.invoke("Sag exakt: pong")
    print("Antworttyp:", type(msg))
    # msg ist i.d.R. ein AIMessage – gib Inhalt sicher aus:
    print("Inhalt:", getattr(msg, "content", msg))
except Exception as e:
    print("FEHLER beim LLM-Aufruf:", repr(e))


C:\Users\amuel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Sende Test-Ping...
Antworttyp: <class 'langchain_core.messages.ai.AIMessage'>
Inhalt: pong


In [2]:
import os
import langchain
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI


#LLM_MODEL = "openai/gpt-oss-20b:free"
LLM_MODEL = "openai/gpt-oss-120b"
LLM_TEMPERATURE = 0.4
BASE_URL = "https://api.groq.com/openai/v1"
OPENROUTER_API_KEY = os.getenv("GROQ_API_KEY_BAI")
USER_PROMPT="Ich verstehe GenAI nicht, kannst du das mir einfach erklären?"

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate


llm = ChatOpenAI(
    model=LLM_MODEL,
    temperature=LLM_TEMPERATURE,
    base_url=BASE_URL,
    api_key=OPENROUTER_API_KEY,
)

print(type(llm))


<class 'langchain_openai.chat_models.base.ChatOpenAI'>


#### Kurz sicherstellen, ob API Key funktioniert

In [4]:
try:
    print(llm.invoke("Sag nur: pong").content)
except Exception as e:
    print(repr(e))

# Test 2: Env-Variablen sichtbar?
import os
print("OPENAI_API_KEY" in os.environ, os.environ.get("OPENAI_BASE_URL"))
print("GROQ_API_KEY_BAI" in os.environ)
print("OPENROUTER_API_KEY" in os.environ)

pong
False None
True
False


#### ChatPrompt Template

In [5]:
from langchain_core.prompts import ChatPromptTemplate
 
LERNASSISTENT_PROMPT = ChatPromptTemplate.from_messages([
    (
        "system",
        (
            "Sprache: Deutsch. Rolle: FHNW-BAI-Lernassistent; erkläre wie eine geduldige Lehrperson.\n"
            "Nutze AUSSCHLIESSLICH den bereitgestellten CONTEXT (Folien/Skripte).\n"
            "Wenn Informationen fehlen oder die Frage nicht im CONTEXT abgedeckt ist, antworte exakt:\n"
            "\"Dazu habe ich im bereitgestellten Material nichts.\" \n"
            "Schlage danach präzise nächste Schritte vor (z. B. welche Folie/Abschnitt hochzuladen wäre).\n"
            "Ziel: Studierende effizient auf Prüfungen vorbereiten.\n"
            "Stil: aktiv, konkret, ohne Floskeln, keine Gender-Sonderzeichen (nutze z. B. 'Lehrperson').\n"
            "Gib GENAU EINEN Lösungsvorschlag und EIN einfaches Beispiel.\n"
            "Halte dich an Terminologie aus dem CONTEXT. Keine externen Fakten, keine Spekulation.\n"
            "CONTEXT:\n{context}"
        )
    ),
    (
        "human",
        (
            "FRAGE: {question}\n"
            "Erstelle die Antwort in genau dieser Struktur:\n"
            "1) Kurzantwort (2–3 Sätze, prüfungsrelevant)\n"
            "2) Erklärung (max. 8 Sätze, schrittweise, mit Intuition)\n"
            "3) Beispiel (sehr einfach, kleine Zahlen/konkreter Mini-Fall)\n"
            "4) Typische Prüfungsfehler (Bullets)\n"
            "5) Verständnis-Check (1–2 Kontrollfragen)\n"
            "6) Quellen (Dokumenttitel + Seiten/Abschnitt aus CONTEXT)"
        )
    ),
])
 

#### PDF Loader

In [6]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyMuPDFLoader  
from pathlib import Path
 
pdf_dir = Path("data/pdfs") #zeigt wo die PDFs gespeichert sind
 
pdf_files = [
    "KI Ueberblick Teil 1.pdf",
    "KI Ueberblick Teil 2.pdf",
    "Problemloesen_als_Suche.pdf",
    "Machine Learning_exam.pdf",
    "Machine Learning.pdf",
    "Wissensrepraesentation.pdf",
    "Aussagenlogik.pdf",
    "Praedikatenlogik.pdf",
    "Deep Learning_exam.pdf",
    "Deep Learning.pdf",
    #"GenAI LLMs.pdf"
]
  
# all_pages_pdf = []
# for name in pdf_files:
#     pdf_path = pdf_dir / name  
#     if not pdf_path.exists():
#         print(f"Datei nicht gefunden: {pdf_path}")
#         continue
#     loader = PyPDFLoader(str(pdf_path))
#     pages = loader.load()
#     all_pages_pdf.extend(pages)
 
# print(f"Loaded {len(all_pages_pdf)} pages from {len(pdf_files)} PDF documents.")
 

all_pages_pdf = []

for name in pdf_files:
    pdf_path = pdf_dir / name
    if not pdf_path.exists():
        print(f"❌ Datei nicht gefunden: {pdf_path}")
        continue

    loader = PyMuPDFLoader(str(pdf_path))
    pages = loader.load()
    all_pages_pdf.extend(pages)
    print(f"✅ {name}: {len(pages)} Seiten geladen")

print(f"\n📚 Insgesamt {len(all_pages_pdf)} Seiten aus {len(pdf_files)} PDF-Dateien geladen.")



✅ KI Ueberblick Teil 1.pdf: 33 Seiten geladen
✅ KI Ueberblick Teil 2.pdf: 72 Seiten geladen
✅ Problemloesen_als_Suche.pdf: 35 Seiten geladen
✅ Machine Learning_exam.pdf: 48 Seiten geladen
✅ Machine Learning.pdf: 78 Seiten geladen
✅ Wissensrepraesentation.pdf: 11 Seiten geladen
✅ Aussagenlogik.pdf: 53 Seiten geladen
✅ Praedikatenlogik.pdf: 26 Seiten geladen
✅ Deep Learning_exam.pdf: 33 Seiten geladen
✅ Deep Learning.pdf: 39 Seiten geladen

📚 Insgesamt 428 Seiten aus 10 PDF-Dateien geladen.


In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# get both websites and pdfs together
all_docs = all_pages_pdf

# define the splitter and strategy
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
splits = splitter.split_documents(all_docs)

In [8]:
import numpy as np

lengths = [len(s.page_content) for s in splits]
print(f"Initial documents: {len(all_docs)}")
print(f"Total chunks: {len(splits)}")
print(f"Avg length: {np.mean(lengths):.1f}")
print(f"Min: {np.min(lengths)}, Max: {np.max(lengths)}")

Initial documents: 428
Total chunks: 971
Avg length: 231.1
Min: 37, Max: 300


In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


C:\Users\amuel\AppData\Local\Temp\ipykernel_3028\4293025327.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


In [10]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embedding_dim = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(embedding_dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
    normalize_L2=True
)

vector_store.add_documents(documents=splits)

['25df0185-1b9f-4cd0-a941-59b3f094f782',
 '99355f05-a008-45aa-bc20-1f4a4bda70df',
 '646d7d3c-1b15-48d5-83cb-442d2f9c49da',
 'a5d62e7b-b373-47ca-b400-c8fd126d7651',
 'e3e38e2b-1412-4d44-904c-761b92dd002f',
 '72c22e61-b2ce-4ed8-a94f-7d202a2ffc7c',
 '5a3d4a8b-8adc-4017-8117-52d32dc2b9af',
 '49054c14-e4d9-445c-a53b-09e2d5d1f228',
 'b86ca8d1-58fd-4c3c-8889-c296e023d7e5',
 'eb45e3f1-9a1a-41a4-af43-9827900565fa',
 '2ba9384d-3480-4025-a650-82405f6baa66',
 'cafceeb3-8ef2-4f2d-a67e-880839cba42a',
 '4897989c-8844-4271-9748-e2daf116b08c',
 'bce47d57-a9e9-4918-9166-408e5b146f53',
 'c4703920-b3b4-477f-936e-f94da156feb4',
 '8bfd73ef-1c29-4eaf-b1fa-eb7216ee8fbd',
 '679e3b7f-0e5f-41fb-981b-3a2d44ee8916',
 'c6c10c68-4049-4843-9305-73d198a6adb9',
 '15496178-d613-4f0e-969b-429fb31e67c1',
 'a9abf7f9-8171-4913-9144-c9c2f4eaf121',
 '6f471fa0-e25c-4a82-bbb8-86fd79451efd',
 'ee19cfb5-68e6-4cba-aeba-45b7879bc42c',
 '60bd2c50-46ec-4db1-92b4-b6331eb42f1f',
 '06a3f40f-7a7c-4ae6-8ef6-b14d67e2e195',
 'bb9138d1-ca0f-

In [11]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [12]:
# docs = retriever.get_relevant_documents("Prädikatenlogik")
# for i, d in enumerate(docs, 1):
#     print(f"--- Retrieved doc {i} ---")
#     print(d.page_content[:400], "...\n")


retriever = vector_store.as_retriever(search_kwargs={"k": 5})
docs = retriever.invoke("Prädikatenlogik")
for i, d in enumerate(docs, 1):
    print(f"\n--- Retrieved doc {i} ---")
    print(d.metadata.get("source"), "p.", d.metadata.get("page"))
    print(d.page_content[:400], "...\n")


--- Retrieved doc 1 ---
data\pdfs\Aussagenlogik.pdf p. 1
Prof. Dr. Knut Hinkelmann
2
Logik…
…korrektes, folgerichtiges Denken ...


--- Retrieved doc 2 ---
data\pdfs\Praedikatenlogik.pdf p. 16
Prof. Dr. Knut Hinkelmann
– Die semantische Folgerung in Prädikatenlogik ist definiert wie in der Aussagenlogik
Semantische Folgerung
17
(Lämmel & Cleve 2023, S. 57)
Sei X eine Menge prädikatenlogischer Formeln, Y eine prädikatenlogische Formel.
Y ist eine semantische Folgerung von X
oder ...


--- Retrieved doc 3 ---
data\pdfs\Praedikatenlogik.pdf p. 12
– Prädikatsymbole werden auf Relationen R der Bezugswelt abgebildet
Iprädikat : P → R
Wahrheitswert prädikatenlogischer Formeln
13
Lämmel & Cleve 2021, S. 56f),
Beispiele: Sei der Gegenstandsbereich eine Menge von Personen
 
mensch(sokrates) ...


--- Retrieved doc 4 ---
data\pdfs\Praedikatenlogik.pdf p. 11
Prof. Dr. Knut Hinkelmann
– Sei in der realen Welt, Knut der Vater von Jens
– Welches der beiden Prädikate ist korrekt?
– vater(knut,jens)
–

In [13]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()


chain = (
{
    "context": retriever,
    "question": RunnablePassthrough(),
}
    | LERNASSISTENT_PROMPT
    | llm
    | StrOutputParser()
)


In [14]:
from langchain_core.output_parsers import StrOutputParser

# USER_PROMPT="Generiere mir einen Post für die PubTour am 16. Oktober"

# result = chain.invoke("Generiere mir einen Post für die PubTour am 16. Oktober")
# print(result)

result = chain.invoke("Was ist Prädikatenlogik?")
print(result)

#result = chain.invoke(user_prompt)
#print(result)

**1) Kurzantwort**  
Die Prädikatenlogik erweitert die Aussagenlogik, indem atomare Formeln aus einem Prädikatssymbol und Termen (Argumenten) bestehen und Quantoren (∀ für „alle“, ∃ für „es existiert“) zulassen. Sie ermöglicht die formale Beschreibung von Aussagen über Objekte und deren Beziehungen.

**2) Erklärung**  
1. In der Prädikatenlogik wird jede atomare Formel durch ein Prädikat p mit n Argumenten (Termen) gebildet (z. B. p(t₁,…,tₙ)).  
2. Prädikatsymbole werden auf Relationen R der Bezugswelt abgebildet (Iₚᵣₑdᵢₖₐₜ : P → R).  
3. Terme können Konstanten, Variablen oder Funktionssymbole sein.  
4. Durch Quantoren können Aussagen über alle oder einige Objekte der Domäne formuliert werden (∀x φ, ∃x φ).  
5. Die Syntax definiert, welche Kombinationen von Prädikaten, Termen und Quantoren zulässig sind.  
6. Die Semantik ordnet jedem Prädikat eine Relation zu und bewertet Formeln als wahr oder falsch in einer Interpretation.  
7. Damit lässt sich die Gültigkeit (semantische Folgerun

## UI

In [15]:
import gradio as gr

def answer(question: str) -> str:
    # Kein LangSmith, einfach direkt die Chain ausführen
    try:
        response = chain.invoke(question)
        return response
    except Exception as e:
        return f"⚠️ Fehler bei der Verarbeitung: {e}"

# --- Gradio UI ---
demo = gr.Interface(
    fn=answer,
    inputs=gr.Textbox(label="Question", placeholder="Type your question here..."),
    outputs=gr.Textbox(label="Answer", lines=10),
    title="Lernassistent BAI",
    description="Stelle Fragen zum Lernstoff der BAI und erhalte präzise, prüfungsrelevante Antworten.",
)

# if __name__ == "__main__":
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=56a0a349-7f2e-43e5-8d52-5ded467f6e9c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>